# Import libs and define functions

In [175]:
import pandas as pd
import subprocess
import os
import shutil
import re
import sys 

In [176]:
def remove_end_in_name(filename):
    splitted = filename.split("_")
    splitted.pop()
    joined_string = "_".join(splitted)
    return joined_string

In [177]:
def get_creation_time(file_path):
    try:
        creation_time = os.path.getctime(file_path)
        return pd.to_datetime(creation_time, unit='s')
    except Exception as e:
        print(f"Error getting creation time for {file_path}: {e}")
        return None

In [178]:
def get_the_table_data():
    for f in os.listdir():
        if f.endswith(".csv"):
                video_schnitt_df = pd.read_csv(f)
                print(f"Table {f} loaded succesfully")
                return video_schnitt_df
    raise ValueError("There is no csv in this directory")

In [179]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [180]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [181]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c:v','copy', #libx264 to encode and copy to test
        '-c:a', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [182]:
def create_folder(folder_name):
    try:
        # Create a new folder in the current working directory
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists.")

In [183]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'libx264', output_file # libx264 to encode and copy to test (but that didnt work the last time)
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [184]:
def get_audio(input_file,output_file,cut_head,cut_tail):
    
    #Construct command
    command = ['ffmpeg',
               '-i', input_file,
               '-ss', cut_head,
               '-to', cut_tail,
               '-vn',
               '-c:a', 'copy', output_file]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [185]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'copy',
               '-c:a', 'copy', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [186]:
def get_columns_to_cut(df):
    pattern = re.compile(r'^rausschneiden\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [187]:
def get_list_of_evens(input_list):
    dummy = []
    for i in range(len(input_list)):
        if i % 2 == 0:
            dummy.append(i)
    return dummy

In [188]:
def concatenate_videos(input_file, output_file):
    
    command = [
        'ffmpeg',
        '-f', 'concat', 
        '-i', input_file,
         '-c:v', 'copy', #libx264 to encode if needed
         '-c:a', 'copy',
        output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occured:", e)

In [189]:
def check_for_non_zero_digit(string):
    # Define a regular expression pattern to match any digit other than 0
    pattern = re.compile('[1-9]')

    if pattern.search(string):
        return True       
    else:
        return False


In [190]:
def extract_data_from_row(df,row_index,columns):
    dic = df.loc[row_index,columns].to_dict()

    return dic

In [237]:
def check_special_case(df):
    columns = get_columns_to_cut(video_schnitt_df)
    for idx, video_name in df["dateiname"].items():
        data = video_schnitt_df["rausschneiden1_ab"][idx]
        if check_for_non_zero_digit(data) == True:
            dic = extract_data_from_row(df,idx,columns)
            counter = 0
            for key,value in dic.items():
                if value == "nan":
                    counter += 1
            if counter < 2:
                print(f"""Error because in row of {video_name}
we need to have at least 2 free 'rausschneiden...' columns 
to execute the code. Please change the csv as needed and
restart the script""")
                sys.exit("Script terminated due to error.")

# Get file and clean it

In [202]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)


Table Videoschnitt_Liste - Spaltennamen (1).csv loaded succesfully
Table Columns cleaned


# Check Special Cases and input error

In [238]:
check_special_case(video_schnitt_df)

In [260]:
def solve_special_case(df):
    
    for idx,video_name in df["dateiname"].items():
        
        if check_for_non_zero_digit(df["rausschneiden1_ab"][idx]) == False:
            continue

        else:
            columns = get_columns_to_cut(df)
            dic = extract_data_from_row(df,idx,columns)
            dummy = []
            counter = 0

            # put values from row into list and change first two columns to 000000
            for key, value in dic.items():
                if value != "nan":
                    dummy.append(value)
                if counter < 2:
                    dic[key] = "00:00:00"
                    counter += 1

        for key, value in dic.items():
            if value != "00:00:00" and dummy:
                dic[key] = dummy.pop(0)
        print(dic)
      

In [261]:
solve_special_case(video_schnitt_df)

{'rausschneiden1_ab': '00:00:00', 'rausschneiden1_bis': '00:00:00', 'rausschneiden2_ab': '00:03:00', 'rausschneiden2_bis': '00:05:00', 'rausschneiden3_ab': 'nan', 'rausschneiden3_bis': 'nan', 'rausschneiden4_ab': 'nan', 'rausschneiden4_bis': 'nan', 'rausschneiden5_ab': 'nan', 'rausschneiden5_bis': 'nan', 'rausschneiden6_ab': 'nan', 'rausschneiden6_bis': 'nan', 'rausschneiden7_ab': 'nan', 'rausschneiden7_bis': 'nan', 'rausschneiden8_ab': 'nan'}
{'rausschneiden1_ab': '00:00:00', 'rausschneiden1_bis': '00:00:00', 'rausschneiden2_ab': '00:02:00', 'rausschneiden2_bis': '00:05:00', 'rausschneiden3_ab': '00:05:00', 'rausschneiden3_bis': '00:05:00', 'rausschneiden4_ab': '00:05:00', 'rausschneiden4_bis': '00:05:00', 'rausschneiden5_ab': '00:05:00', 'rausschneiden5_bis': '00:05:00', 'rausschneiden6_ab': '00:05:00', 'rausschneiden6_bis': '00:05:00', 'rausschneiden7_ab': '00:05:00', 'rausschneiden7_bis': '00:05:00', 'rausschneiden8_ab': '00:05:00'}


In [209]:
for key, value in dic.items():
    if value != "00:00:00":
        try:
            dic[key] = dummy.pop(0)
        except Exception as error:
            print("Error occured:",error)
            break
print(dic)

Error occured: pop from empty list
{'rausschneiden1_ab': '00:00:00', 'rausschneiden1_bis': '00:00:00', 'rausschneiden2_ab': '00:03:00', 'rausschneiden2_bis': '00:05:00', 'rausschneiden3_ab': 'nan', 'rausschneiden3_bis': 'nan', 'rausschneiden4_ab': 'nan', 'rausschneiden4_bis': 'nan', 'rausschneiden5_ab': 'nan', 'rausschneiden5_bis': 'nan', 'rausschneiden6_ab': 'nan', 'rausschneiden6_bis': 'nan', 'rausschneiden7_ab': 'nan', 'rausschneiden7_bis': 'nan', 'rausschneiden8_ab': 'nan'}


In [210]:
for idx, video_name in video_schnitt_df["dateiname"].items():
    data = video_schnitt_df["rausschneiden1_ab"][idx]
    if check_for_non_zero_digit(data) == True:
        print(video_name,"neues if statement")

Normalverteilung.mp4 neues if statement
ELR_Geradengleichung_Einleitung.mp4 neues if statement


# Get Standbild

In [464]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        output_file = f"{video_name.split('.')[0]}_standbild.mp4"
        cut_head = video_schnitt_df["standbild_bis"][idx]

        #building logic when just to cut head or tail
        if cut_head == "nan":
            continue
        print("video_name:",video_name,"cuttime:",cut_head,)
        get_standbild(video_name,output_file,cut_head)
except Exception as error:
    print("An error occurred:", error)



video_name: ELR_Geradengleichung.mp4 cuttime: 00:02:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video_name: ELR_Geradengleichung_Einleitung.mp4 cuttime: 00:01:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

[libx264 @ 0x7fdebba08c00] using SAR=1/1
[libx264 @ 0x7fdebba08c00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7fdebba08c00] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x7fdebba08c00] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=6 lookahead_threads=1 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'ELR_Geradengleichung_Einleitung_standbild.mp4':
  Metadata:
    major_brand     : isom


# Get audio

In [465]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue

        #Define arguments
        output_file = f"{video_name.split('.')[0]}_head_audio.mp4"
        cut_head = "00:00:00"
        cut_tail = video_schnitt_df["standbild_bis"][idx]

        print("video_name",video_name,"output_file", output_file,"cut_head",cut_head,"cut_tail:", cut_tail)
        get_audio(video_name,output_file,cut_head,cut_tail)

except Exception as error:
    print("An error occurred:", error)

video_name ELR_Geradengleichung.mp4 output_file ELR_Geradengleichung_head_audio.mp4 cut_head 00:00:00 cut_tail: 00:02:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video_name ELR_Geradengleichung_Einleitung.mp4 output_file ELR_Geradengleichung_Einleitung_head_audio.mp4 cut_head 00:00:00 cut_tail: 00:01:00


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

# Cut head of the original Video

In [466]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        # define arguments
        cut_head = video_schnitt_df["standbild_bis"][idx]
        cut_tail = get_video_duration(video_name)
        output_file = f"{video_name.split('.')[0]}_ohne_start_standbild.mp4"
    
        print("input name:",video_name,"Schnittpunkte:", cut_head, cut_tail)
        cut_head_tail(video_name, output_file, cut_head, cut_tail)
except Exception as error:
    print("An error occurred:", error)

input name: ELR_Geradengleichung.mp4 Schnittpunkte: 00:02:00 00:14:42.88


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

input name: ELR_Geradengleichung_Einleitung.mp4 Schnittpunkte: 00:01:00 00:14:42.93


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

# Merge audio and video

In [467]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        video_file = f"{video_name.split('.')[0]}_standbild.mp4"
        audio_file = f"{video_name.split('.')[0]}_head_audio.mp4"
        output_file = f"{video_name.split('.')[0]}_merged_start.mp4"
        
        print("video:", video_file, "audio:", audio_file, "output", output_file)
        merge_audio_and_video(video_file,audio_file,output_file)

except Exception as error:
    print("An error occured:", error)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

video: ELR_Geradengleichung_standbild.mp4 audio: ELR_Geradengleichung_head_audio.mp4 output ELR_Geradengleichung_merged_start.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'ELR_Geradengleichung_standbild.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.3.100
  Duration: 00:00:00.03, start: 0.000000, bitrate: 5120 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 4913 kb/s, 30 fps, 30 tbr, 15360 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.3.100 libx264
Input #1, mov,mp4,m4a,3gp,3g2,mj2, from 'ELR_Geradengleichung_head_audio.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2mp41
    encoder         : Lavf60.3.100
  Duration: 00:02:00.02, start: 0.000000, bitrate: 193 kb/s
  Stream #1:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 191 kb/s (default)
    Metadata:
      handler_name 

video: ELR_Geradengleichung_Einleitung_standbild.mp4 audio: ELR_Geradengleichung_Einleitung_head_audio.mp4 output ELR_Geradengleichung_Einleitung_merged_start.mp4


frame=    1 fps=0.0 q=-1.0 Lsize=    1448kB time=00:00:59.98 bitrate= 197.8kbits/s speed=1.96e+03x    
video:30kB audio:1406kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.862874%


# Concat to endproduct

In [468]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        #define input for the text file with the videos we want to concat
        video1 = f"{video_name.split('.')[0]}_merged_start.mp4"
        video2 = f"{video_name.split('.')[0]}_ohne_start_standbild.mp4"
        textfile_content = f"file '{video1}'\nfile '{video2}'"

        #create the file with content
        with open('dummy.txt', 'w') as textfile:
            textfile.write(textfile_content)

        #define parameters for concat function
        output_file = f"{video_name.split('.')[0]}_standbild_endprodukt.mp4"
        textfile_name = "dummy.txt"
        print("file content:", textfile_content)
        concatenate_videos(textfile_name,output_file)
        
except Exception as error:
    print("An error occured:", error)

file content: file 'ELR_Geradengleichung_merged_start.mp4'
file 'ELR_Geradengleichung_ohne_start_standbild.mp4'


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

file content: file 'ELR_Geradengleichung_Einleitung_merged_start.mp4'
file 'ELR_Geradengleichung_Einleitung_ohne_start_standbild.mp4'


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7ff78580fb00] Could not find codec parameters for stream 0 (Video: h264 (avc1 / 0x31637661), none(tv, bt709), 1920x1080, 197 kb/s): unspecified pixel format
Consider increasing the value for the 'analyzeduration' (0) and 'probesize' (5000000) options
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7ff78580fb00] Auto-inserting h264_mp4toannexb bitstream filter
frame=24510 fps=18712 q=-1.0 Lsize=   41286kB time=00:14:42.85 bitrate= 383.1kbits/s speed= 674x    
video:19701kB audio:20692kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 2.209858%


# Create folder and move files

In [482]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

Folder 'endprodukte' already exists.
Folder 'script_output' already exists.


In [470]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [471]:
for filename in os.listdir():
    if filename.endswith("endprodukt.mp4"):
        shutil.move(filename,path_folder_endprodukte)
    elif filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_script_output)

# Start to cut things out

In [33]:
# cut all the videos that did not use the standbild ersetzen script
try:
    # videos need to be in same directory with python script
    trash_list = []
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if not os.path.exists(video_name):
            print("Following file does not exist:",video_name)
            continue

        #check ob video schon standbid ersetzt bekommen, use other videoname
        if video_schnitt_df["standbild_bis"][idx] != "nan":
            print(video_name,"skipped because other video in endprodukte needs to be used")
            continue

        columns_to_cut = get_columns_to_cut(video_schnitt_df)
        evens = get_list_of_evens(columns_to_cut)

        parts_list = []
        video1 = 0
        video2 = 0
        end_reached = None
        print("Video Name to cut:", video_name)

        try:
            for i,col in enumerate(columns_to_cut):
                cut_head = video_schnitt_df[col][idx]
                cut_tail = video_schnitt_df[columns_to_cut[i+1]][idx]
                print("index:",i,"Column:",col)
                print("Timestamps before changing:",cut_head,cut_tail)

                #logic to handle the nans 
                
                if cut_head != "nan" and cut_head != "00:00:00" and i == 0:
                    print("Skipped because of special case")
                    
                    break
                
                if cut_head == "nan" and cut_tail == "nan":
                    print("Break used")
                    break
                
                if end_reached == True:
                    break

                if  cut_head == "nan":
                    cut_head = "00:00:00"
                    print("cut_head ersetzt")

                if cut_tail == "nan":
                    cut_tail = get_video_duration(video_name)
                    end_reached = True
                    print("end reached")
                    
                output_file = f"{video_name.split('.')[0]}_part{i}.mp4"
                print("Timestamps after changing:",cut_head,cut_tail)
                #cut_head_tail(video_name, output_file, cut_head, cut_tail)
                
                if i not in evens:
                    parts_list.append(output_file)
                else:
                    trash_list.append(output_file)

                if len(parts_list) == 2:
                    video1 = parts_list[0]
                    video2 = parts_list[1]
                    
                    textfile_content = f"file '{video1}'\nfile '{video2}'"

                    #create the file with content
                    with open('dummy.txt', 'w') as textfile:
                        textfile.write(textfile_content)

                    #define parameters for concat function
                    textfile_name = "dummy.txt"

                    output_file = f"{video_name.split('.')[0]}_concatted{i}.mp4"
                    
                    parts_list.clear()
                    parts_list.append(output_file)
                    print(textfile_content, "output:", output_file)
                    #concatenate_videos(textfile_name,output_file)                

        except Exception as e:
            print("skip to next video because:",e)

except Exception as error:
    print("An error occured:", error)   

Video Name to cut: Normalverteilung.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: 00:03:00 00:05:00
Skipped because of special case
index: 1 Column: rausschneiden1_bis
Timestamps before changing: 00:05:00 nan
end reached
Timestamps after changing: 00:05:00 00:50:46.08
index: 2 Column: rausschneiden2_ab
Timestamps before changing: nan nan
Break used
Video Name to cut: ELR_Geradengleichung.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan 00:01:44.000
cut_head ersetzt
Timestamps after changing: 00:00:00 00:01:44.000
index: 1 Column: rausschneiden1_bis
Timestamps before changing: 00:01:44.000 00:02:04.000
Timestamps after changing: 00:01:44.000 00:02:04.000
index: 2 Column: rausschneiden2_ab
Timestamps before changing: 00:02:04.000 00:02:08.000
Timestamps after changing: 00:02:04.000 00:02:08.000
index: 3 Column: rausschneiden2_bis
Timestamps before changing: 00:02:08.000 00:10:35.500
Timestamps after changing: 00:02:08.000 00:10:35.500
file 'ELR_

## cut all the videos that did not use the standbild ersetzen script


In [68]:

try:
    # videos need to be in same directory with python script
    trash_list = []
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if not os.path.exists(video_name):
            print("Following file does not exist:",video_name)
            continue

        #check ob video schon standbid ersetzt bekommen, use other videoname
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            print(f"skipped this {video_name} because it is already finished")
            continue
        
        folder_name = "endprodukte/"
        new_video_name = folder_name + f"{video_name.split('.')[0]}_standbild_endprodukt.mp4"

        columns_to_cut = get_columns_to_cut(video_schnitt_df)
        evens = get_list_of_evens(columns_to_cut)

        parts_list = []
        video1 = 0
        video2 = 0
        end_reached = None
        print("Video Name to cut:", video_name, "actual input to cut:", new_video_name)

        try:
            for i,col in enumerate(columns_to_cut):
                cut_head = video_schnitt_df[col][idx]
                cut_tail = video_schnitt_df[columns_to_cut[i+1]][idx]
                print("index:",i,"Column:",col)
                print("Timestamps before changing:",cut_head,cut_tail)

                #logic to handle the nans
                if cut_head != "nan" and cut_head != "00:00:00" and i == 0:
                    print("Skipped because of special case")
                    break
                
                if cut_head == "nan" and cut_tail == "nan":
                    print("Break used")
                    break
                
                if end_reached == True:
                    print("Break used")
                    break

                if  cut_head == "nan":
                    cut_head = "00:00:00"
                    print("cut_head ersetzt")

                if cut_tail == "nan":
                    cut_tail = get_video_duration(video_name)
                    end_reached = True
                    print("end reached")
                    
                output_file = f"{video_name.split('.')[0]}_part{i}.mp4"
                print("Timestamps after changing:",cut_head,cut_tail)
                #cut_head_tail(new_video_name, output_file, cut_head, cut_tail)
                
                if i not in evens:
                    parts_list.append(output_file)
                else:
                    trash_list.append(output_file)

                if len(parts_list) == 2:
                    video1 = parts_list[0]
                    video2 = parts_list[1]
                    
                    textfile_content = f"file '{video1}'\nfile '{video2}'"

                    #create the file with content
                    with open('dummy.txt', 'w') as textfile:
                        textfile.write(textfile_content)

                    #define parameters for concat function
                    textfile_name = "dummy.txt"

                    output_file = f"{video_name.split('.')[0]}_concatted{i}.mp4"
                    
                    parts_list.clear()
                    parts_list.append(output_file)
                    print(textfile_content, "output:", output_file)
                    #concatenate_videos(textfile_name,output_file)                

        except Exception as e:
            print("skip to next video because:",e)

except Exception as error:
    print("An error occured:", error)   

skipped this Normalverteilung.mp4 because it is already finished
skipped this ELR_Geradengleichung.mp4 because it is already finished
Video Name to cut: ELR_Geradengleichung_Einleitung.mp4 actual input to cut: endprodukte/ELR_Geradengleichung_Einleitung_standbild_endprodukt.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: 00:03:00 00:05:00
Skipped because of special case
Video Name to cut: Normalverteilung2.mp4 actual input to cut: endprodukte/Normalverteilung2_standbild_endprodukt.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan 00:01:44.000
cut_head ersetzt
Timestamps after changing: 00:00:00 00:01:44.000
index: 1 Column: rausschneiden1_bis
Timestamps before changing: 00:01:44.000 00:02:04.000
Timestamps after changing: 00:01:44.000 00:02:04.000
index: 2 Column: rausschneiden2_ab
Timestamps before changing: 00:02:04.000 00:02:08.000
Timestamps after changing: 00:02:04.000 00:02:08.000
index: 3 Column: rausschneiden2_bis
Timestamps before changi

# Rename the right videos to _endprodukt

In [474]:
# create a list with videonames of the youngest version that is not in trash list
files = []

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    youngest_creation_time = None
    youngest_video = None 

    for f in os.listdir():     
        if f.endswith(".mp4") and f not in trash_list:
            if "_" in f:
                part_prefix = remove_end_in_name(f)
            else:
                part_prefix = f.split('.')[0]
            
            if part_prefix == original_prefix:
                creation_time = get_creation_time(f)
                if youngest_creation_time is None or creation_time > youngest_creation_time:
                    youngest_creation_time = creation_time
                    youngest_video = f     
    if youngest_video is not None:
        files.append(youngest_video)    
  
print("All the youngest versions of a video:",files)

All the youngest versions of a video: ['Normalverteilung.mp4', 'ELR_Geradengleichung_concatted5.mp4', 'ELR_Geradengleichung_Einleitung_part1.mp4', 'Normalverteilung2.mp4']


In [475]:
# create a list of original video_names
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [476]:
#Compare the filenames with the original Videos
files_to_reaname = [element for element in files if element not in original_video_names]

print("Files that are gonna be renamed:",files_to_reaname)

Files that are gonna be renamed: ['ELR_Geradengleichung_concatted5.mp4', 'ELR_Geradengleichung_Einleitung_part1.mp4']


In [477]:
# rename the videos
try:
    for file in files_to_reaname:
        new_name = f"{file.split('.')[0]}_cut_things_out_endprodukt.mp4"
        os.rename(file,new_name)
        print(new_name)
except Exception as e:
    print("Error while renaming:",e)

ELR_Geradengleichung_concatted5_cut_things_out_endprodukt.mp4
ELR_Geradengleichung_Einleitung_part1_cut_things_out_endprodukt.mp4


# Move files

In [478]:
for filename in os.listdir():
    if filename.endswith("endprodukt.mp4"):
        shutil.move(filename,path_folder_endprodukte)
    elif filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_script_output)

# Delete old endproducts

In [479]:
# create a list with videonames of the youngest version that is not in trash list
files = []

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    youngest_creation_time = None
    youngest_video = None 
    pattern = re.compile(rf'^{original_prefix}_.*$')

    for f in os.listdir("endprodukte/"):     
        
        if pattern.match(f):
            creation_time = get_creation_time("endprodukte/"+f)
            if youngest_creation_time is None or creation_time > youngest_creation_time:
                youngest_creation_time = creation_time
                youngest_video = f     
    if youngest_video is not None:
        files.append(youngest_video)    
  
print("All the youngest versions of a video:",files)

All the youngest versions of a video: ['ELR_Geradengleichung_concatted5_cut_things_out_endprodukt.mp4', 'ELR_Geradengleichung_Einleitung_part1_cut_things_out_endprodukt.mp4']


In [480]:
remove_list = []

for f in os.listdir("endprodukte/"):     
    if f not in files:
        remove_list.append(f)

print(remove_list)

['ELR_Geradengleichung_standbild_endprodukt.mp4', 'ELR_Geradengleichung_Einleitung_standbild_endprodukt.mp4']


In [481]:
for i in remove_list:
    os.remove("endprodukte/" + i)